In [1]:
%config Completer.use_jedi = False

In [2]:
import os
from tqdm.notebook import tqdm
from skimage import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import albumentations
import torch

from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sys
sys.path.append('../')

from utils.general import *
import utils.dataload as d
from models import model_selector
from utils.data_augmentation import data_augmentation_selector
from medpy.metric.binary import hd, dc, jc, assd

from utils.neural import *
from utils.datasets import *
from utils.metrics import *

In [4]:
def find_path(directory, filename):
    for path in Path(directory).rglob(filename):
        return path

In [5]:
def save_pred(image, mask, pred_mask, case, metric_value, descriptor):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(17, 10))
    fig.tight_layout(pad=3)  # Set spacing between plots

    ax1.imshow(image, cmap="gray")
    ax1.axis("off")
    ax1.set_title("Input Image")

    masked_lv = np.ma.masked_where(mask == 0, mask)
    ax2.imshow(image, cmap="gray")
    ax2.imshow(masked_lv, 'hsv', interpolation='bilinear', alpha=0.33)
    ax2.axis("off")
    ax2.set_title("Ground-truth")

    masked_lv = np.ma.masked_where(pred_mask == 0, pred_mask)
    ax3.imshow(image, cmap="gray")
    ax3.imshow(masked_lv, 'hsv', interpolation='bilinear', alpha=0.33)
    ax3.axis("off")
    ax3.set_title("Automatic Segmentation")

    fig.suptitle(f"{case} - Jaccard {metric_value:.4f}", y=0.9)
    parent_dir = os.path.join("CrossDatabase_v2/ALLMMS2ACDC3D_preds_overlays", descriptor)
    os.makedirs(parent_dir, exist_ok=True)
    plt.savefig(os.path.join(parent_dir, f"{case}.jpg"), dpi=300)
    plt.close()

In [6]:
value_ranges = [0, 0.25, 0.5, 0.75, 1]
values_desc = ["awful", "average", "good", "excellent"]

for i in range(4):
    print(f"{value_ranges[i]} - {value_ranges[i+1]}: {values_desc[i]}")

0 - 0.25: awful
0.25 - 0.5: average
0.5 - 0.75: good
0.75 - 1: excellent


In [7]:
model = model_selector(
    "segmentation", "resnet34_unet_imagenet_encoder_scse_hypercols", num_classes=4, from_swa=True,
    in_channels=3, devices=[0], checkpoint="../checks/ACDC/n1_100_swa.pt"
)

_, _, val_aug = data_augmentation_selector(
    "lvsc2d", 224, 224, "padd"
)


--- Frosted pretrained backbone! ---
Model total number of parameters: 35749488
Loaded model from checkpoint: ../checks/ACDC/n1_100_swa.pt
Using LVSC 2D Segmentation Data Augmentation Combinations
Padding masks!
Padding masks!


In [8]:
batch_size = 1
add_depth = True
normalization = "standardize"

train_dataset = ALLMMsDataset3D(
    transform=val_aug, img_transform=[],
    add_depth=add_depth, normalization=normalization, data_relative_path="../../mnms_da"
)

allmms_loader =  DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True,
    drop_last=False, collate_fn=train_dataset.simple_collate
)

In [9]:
train_dataset.df

,External code,VendorName,Vendor,Centre,ED,ES,Partition,Labeled
0,A0S9V9,Siemens,A,1,0,9,Training,True
1,A1D0Q7,Philips,B,2,0,9,Training,True
2,A1D9Z7,Siemens,A,1,22,11,Training,True
3,A1E9Q1,Siemens,A,1,0,9,Training,True
4,A1K2P5,Canon,D,5,33,11,Testing,False
...,...,...,...,...,...,...,...,...
315,T2Z1Z9,Canon,D,5,29,9,Testing,False
316,T9U9W2,Siemens,A,1,0,10,Training,True
317,V4W8Z5,GE,C,4,19,9,Testing,False
318,W5Z4Z8,Philips,B,2,29,11,Training,True


In [10]:
def map_mask_classes(mask, classes_map):
    """

    Args:
        mask: (np.array) Mask Array to map (height, width)
        classes_map: (dict) Mapping between classes. E.g.  {0:0, 1:3, 2:2, 3:1 ,4:4}

    Returns: (np.array) Mapped mask array

    """
    res = np.zeros_like(mask).astype(mask.dtype)
    for value in np.unique(mask):
        if value not in classes_map:
            assert False, f"Please specify all class maps. {value} not in {classes_map}"
        res += np.where(mask == value, classes_map[value], 0).astype(mask.dtype)
    return res

In [11]:
model.eval()

metrics = {
    'img_id':[], 'phase':[],
    'iou_RV':[], 'dice_RV':[], 'hd_RV':[], 'assd_RV': [], 
    'iou_MYO':[], 'dice_MYO':[], 'hd_MYO':[], 'assd_MYO': [],
    'iou_LV':[], 'dice_LV':[], 'hd_LV':[], 'assd_LV': [],
}
plot_per_range = [50,50,50,50] # ["awful", "average", "good", "excellent"]
preds_dir = "CrossDatabase_v2/ACDCD2ALLMMS3D"
os.makedirs(preds_dir, exist_ok=True)

# MMS -> class_to_cat = {1: "LV", 2: "MYO", 3: "RV"}
# Original ACDC -> class_to_cat = {1: "RV", 2: "MYO", 3: "LV"}
map_classes = {0: 0, 1: 3, 2: 2, 3: 1}
class_to_cat = {1: "RV", 2: "MYO", 3: "LV"}
mask_reshape_method = "padd"
include_background = False

with torch.no_grad():
    for batch_indx, batch in enumerate(tqdm(allmms_loader)):
        img_id = batch["external_code"]
        partition = batch["partition"]
        init_shape = batch["initial_shape"]
        
        ed_vol = batch["ed_volume"].squeeze().cuda()
        es_vol = batch["es_volume"].squeeze().cuda()
        
        original_ed = batch["original_ed"]
        original_es = batch["original_es"]
        
        original_ed_mask = batch["original_ed_mask"]
        original_es_mask = batch["original_es_mask"]
        
        mask_affine = batch["affine"]
        mask_header = batch["header"]
        
        save_dir = os.path.join(preds_dir, partition, "Labeled" if partition=="Training" else "", img_id)
        os.makedirs(save_dir, exist_ok=True)
        
        # ---- ED CASE ----
        
        prob_preds = model(ed_vol)
        pred_mask = convert_multiclass_mask(prob_preds).data.cpu().numpy()
        pred_mask = pred_mask.astype(np.uint8)
        pred_mask = map_mask_classes(pred_mask, map_classes)
        pred_mask = reshape_volume(pred_mask, init_shape[:2], mask_reshape_method)
        pred_mask = pred_mask.transpose(1,2,0)
        
        for current_class in range(len(map_classes)):

            if not include_background and current_class == 0:
                continue

            y_true = np.where(original_ed_mask == current_class, 1, 0).astype(np.int32)
            y_pred = np.where(pred_mask == current_class, 1, 0).astype(np.int32)
            class_str = class_to_cat[current_class]

            jc_score = jaccard_coef(y_true, y_pred)
            dc_score = dice_coef(y_true, y_pred)
            hd_score = secure_hd(y_true, y_pred)
            assd_score = secure_assd(y_true, y_pred)

            metrics[f'iou_{class_str}'].append(jc_score)
            metrics[f'dice_{class_str}'].append(dc_score)
            metrics[f'hd_{class_str}'].append(hd_score)
            metrics[f'assd_{class_str}'].append(assd_score)

        metrics[f'img_id'].append(img_id)
        metrics[f'phase'].append("ED")
    
        pred_name = f"{img_id}_ED.nii.gz"
        d.save_nii(os.path.join(save_dir, pred_name), pred_mask, mask_affine, mask_header)
        
        # ---- ES CASE ----
        
        prob_preds = model(es_vol)
        pred_mask = convert_multiclass_mask(prob_preds).data.cpu().numpy()
        pred_mask = pred_mask.astype(np.uint8)
        pred_mask = map_mask_classes(pred_mask, map_classes)
        pred_mask = reshape_volume(pred_mask, init_shape[:2], mask_reshape_method)
        pred_mask = pred_mask.transpose(1,2,0)
        
        for current_class in range(len(map_classes)):

            if not include_background and current_class == 0:
                continue

            y_true = np.where(original_es_mask == current_class, 1, 0).astype(np.int32)
            y_pred = np.where(pred_mask == current_class, 1, 0).astype(np.int32)
            class_str = class_to_cat[current_class]

            jc_score = jaccard_coef(y_true, y_pred)
            dc_score = dice_coef(y_true, y_pred)
            hd_score = secure_hd(y_true, y_pred)
            assd_score = secure_assd(y_true, y_pred)

            metrics[f'iou_{class_str}'].append(jc_score)
            metrics[f'dice_{class_str}'].append(dc_score)
            metrics[f'hd_{class_str}'].append(hd_score)
            metrics[f'assd_{class_str}'].append(assd_score)

        metrics[f'img_id'].append(img_id)
        metrics[f'phase'].append("ES")
    
        pred_name = f"{img_id}_ES.nii.gz"
        d.save_nii(os.path.join(save_dir, pred_name), pred_mask, mask_affine, mask_header)

  0%|          | 0/320 [00:00<?, ?it/s]

In [12]:
df = pd.DataFrame(metrics)
df.head()

,img_id,phase,iou_RV,dice_RV,hd_RV,assd_RV,iou_MYO,dice_MYO,hd_MYO,assd_MYO,iou_LV,dice_LV,hd_LV,assd_LV
0,A0S9V9,ED,0.897830,0.946165,16.673332,0.312530,0.629258,0.772447,16.401219,0.466269,0.822870,0.902829,45.847574,1.732925
1,A0S9V9,ES,0.850786,0.919378,4.123106,0.337256,0.627975,0.771480,7.000000,0.385144,0.677177,0.807520,46.486557,2.634413
2,A1D0Q7,ED,0.867756,0.929196,5.099020,0.255877,0.677817,0.807975,7.874008,0.436800,0.756587,0.861428,23.086793,0.451549
3,A1D0Q7,ES,0.653397,0.790369,27.495454,0.709843,0.718121,0.835938,7.615773,0.544618,0.545095,0.705581,7.141428,0.551595
4,A1D9Z7,ED,0.848323,0.917938,4.000000,0.420525,0.726872,0.841836,7.000000,0.538348,0.774450,0.872890,17.916473,0.526392


## Get metrics by replacing infinite distance values with max value

In [13]:
print(f"Mean IOU RV: {df['iou_RV'].mean()}")
print(f"Mean IOU LV: {df['iou_LV'].mean()}")
print(f"Mean IOU MYO: {df['iou_MYO'].mean()}")

print("--------------")

print(f"Mean DICE RV: {df['dice_RV'].mean()}")
print(f"Mean DICE LV: {df['dice_LV'].mean()}")
print(f"Mean DICE MYO: {df['dice_MYO'].mean()}")

print("--------------")

print(f"Mean Hausdorff RV: {df['hd_RV'].mean()}")
print(f"Mean Hausdorff LV: {df['hd_LV'].mean()}")
print(f"Mean Hausdorff MYO: {df['hd_MYO'].mean()}")

print("--------------")

print(f"Mean ASSD RV: {df['assd_RV'].mean()}")
print(f"Mean ASSD LV: {df['assd_LV'].mean()}")
print(f"Mean ASSD MYO: {df['assd_MYO'].mean()}")

Mean IOU RV: 0.7626529835183558
Mean IOU LV: 0.7325406903908253
Mean IOU MYO: 0.6693232263124008
--------------
Mean DICE RV: 0.8554188564213596
Mean DICE LV: 0.8327970442657217
Mean DICE MYO: 0.7957930515752342
--------------
Mean Hausdorff RV: 18.30224804449799
Mean Hausdorff LV: 15.37283516426593
Mean Hausdorff MYO: 18.50453754740015
--------------
Mean ASSD RV: 1.1926228914281372
Mean ASSD LV: 0.9051902093486305
Mean ASSD MYO: 0.6965292350784242


In [14]:
min_hausdorff_lv = df["hd_LV"].min()
print(f"min_hausdorff_lv: {min_hausdorff_lv}")
min_hausdorff_rv = df["hd_RV"].min()
print(f"min_hausdorff_rv: {min_hausdorff_rv}")
min_hausdorff_myo = df["hd_MYO"].min()
print(f"min_hausdorff_myo: {min_hausdorff_myo}")

min_assd_lv = df["assd_LV"].min()
print(f"min_assd_lv: {min_assd_lv}")
min_assd_rv = df["assd_RV"].min()
print(f"min_assd_rv: {min_assd_rv}")
min_assd_myo = df["assd_MYO"].min()
print(f"min_assd_myo: {min_assd_myo}")

min_hausdorff_lv: -1.0
min_hausdorff_rv: -1.0
min_hausdorff_myo: -1.0
min_assd_lv: -1.0
min_assd_rv: -1.0
min_assd_myo: -1.0


In [15]:
max_hausdorff_lv = df["hd_LV"].max()
max_hausdorff_rv = df["hd_RV"].max()
max_hausdorff_myo = df["hd_MYO"].max()

max_assd_lv = df["assd_LV"].max()
max_assd_rv = df["assd_RV"].max()
max_assd_myo = df["assd_MYO"].max()

In [16]:
print(f"Mean IOU RV: {df['iou_RV'].mean()}")
print(f"Mean IOU LV: {df['iou_LV'].mean()}")
print(f"Mean IOU MYO: {df['iou_MYO'].mean()}")

print("--------------")

print(f"Mean DICE RV: {df['dice_RV'].mean()}")
print(f"Mean DICE LV: {df['dice_LV'].mean()}")
print(f"Mean DICE MYO: {df['dice_MYO'].mean()}")

print("--------------")

print(f"Mean Hausdorff RV: {df['hd_RV'].mean()}")
print(f"Mean Hausdorff LV: {df['hd_LV'].mean()}")
print(f"Mean Hausdorff MYO: {df['hd_MYO'].mean()}")

print("--------------")

print(f"Mean ASSD RV: {df['assd_RV'].mean()}")
print(f"Mean ASSD LV: {df['assd_LV'].mean()}")
print(f"Mean ASSD MYO: {df['assd_MYO'].mean()}")

Mean IOU RV: 0.7626529835183558
Mean IOU LV: 0.7325406903908253
Mean IOU MYO: 0.6693232263124008
--------------
Mean DICE RV: 0.8554188564213596
Mean DICE LV: 0.8327970442657217
Mean DICE MYO: 0.7957930515752342
--------------
Mean Hausdorff RV: 18.30224804449799
Mean Hausdorff LV: 15.37283516426593
Mean Hausdorff MYO: 18.50453754740015
--------------
Mean ASSD RV: 1.1926228914281372
Mean ASSD LV: 0.9051902093486305
Mean ASSD MYO: 0.6965292350784242


In [17]:
df.groupby("phase").mean()

,iou_RV,dice_RV,hd_RV,assd_RV,iou_MYO,dice_MYO,hd_MYO,assd_MYO,iou_LV,dice_LV,hd_LV,assd_LV
phase,,,,,,,,,,,,
ED,0.846114,0.913586,14.560633,0.726751,0.656596,0.788685,17.107950,0.616567,0.792198,0.875884,13.641386,0.632568
ES,0.679192,0.797252,22.043863,1.658495,0.682050,0.802901,19.901125,0.776491,0.672883,0.789710,17.104284,1.177813


In [18]:
df.to_csv(os.path.join(preds_dir, "results.csv"), index=False)